In [1]:
import pandas as pd
import json

In [2]:
data = pd.read_csv("cleaned_resume.csv")
data.head(5)


,Unnamed: 0,clean_content,entities_mapped
0,0,Abhishek Jha Application Development Associate...,"['Name', 'Name', 'Designation', 'Designation',..."
1,1,Afreen Jamadar Active member IIIT Committee Th...,"['Name', 'Name', 'O', 'O', 'O', 'O', 'O', 'O',..."
2,2,Akhil Yadav Polemaina Telangana Email indeed.c...,"['Name', 'Name', 'Name', 'O', 'O', 'Email Addr..."
3,3,Alok Khandai Operational Analyst Engineer UNIS...,"['Name', 'Name', 'Designation', 'Designation',..."
4,4,Ananya Chavan lecturer oracle tutorials Mahara...,"['Name', 'Name', 'Designation', 'Companies wor..."


In [30]:
c2 = data["entities_mapped"].unique
c2
#Name 
#Designation
#Companies worked at
#Email Address
#Skills
#Years of Experience
#Location

<bound method Series.unique of 0      ['Name', 'Name', 'Designation', 'Designation',...
1      ['Name', 'Name', 'O', 'O', 'O', 'O', 'O', 'O',...
2      ['Name', 'Name', 'Name', 'O', 'O', 'Email Addr...
3      ['Name', 'Name', 'Designation', 'Designation',...
4      ['Name', 'Name', 'Designation', 'Companies wor...
                             ...                        
215    ['Name', 'Name', 'Designation', 'O', 'O', 'O',...
216    ['Name', 'Name', 'Designation', 'Designation',...
217    ['Name', 'Name', 'Designation', 'Designation',...
218    ['Name', 'Name', 'Designation', 'Designation',...
219    ['Name', 'Name', 'Designation', 'Designation',...
Name: entities_mapped, Length: 220, dtype: object>


* .word_ids - Use bcz input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a [CLS] and a [SEP] above) and then because of those possible splits of words in multiple tokens:

In [41]:
len(data['clean_content']),len(data['entities_mapped'])

(220, 220)

In [42]:
data.shape
label_all_tokens = True

In [43]:
X=list(data['clean_content'])

In [44]:
y=list(data['entities_mapped'])

In [45]:
#y=list(pd.get_dummies(y,drop_first=True)['entities_mapped'])

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [47]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer =  AutoTokenizer.from_pretrained('dslim/bert-large-NER', do_lower_case = False)

In [48]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [50]:
%%time
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

CPU times: user 294 ms, sys: 26 ms, total: 320 ms
Wall time: 418 ms


In [51]:
train_dataset

<TensorSliceDataset shapes: ({input_ids: (512,), token_type_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.string)>

In [52]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [53]:
from transformers import BertForTokenClassification,TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [54]:
with training_args.strategy.scope():
    model = BertForTokenClassification.from_pretrained("dslim/bert-large-NER")
trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/transformers/trainer_tf.py:109: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  warnings.warn(


Init Plugin
Init Graph Optimizer
Init Kernel


2022-01-06 15:44:21.259672: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
input: "Placeholder/_3"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
      type: DT_STRING
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 512
        }
      }
      shape {
        dim {
          size: 512
        }
      }
      shape {
        dim {
          size: 512
        }
      }
      shape {
      }
    }
  }
}



ValueError: `Checkpoint` was expecting a trackable object (an object derived from `TrackableBase`), got BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linea

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


In [17]:
model = BertForTokenClassification.from_pretrained("dslim/bert-large-NER")

In [53]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

AttributeError: 'BertModel' object has no attribute 'compile'